In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage.transform import resize
import tensorflow as tf
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

from joblib import Parallel, delayed
import time, math
import random
import cv2
#import png

def convert_to_png(dicomimage, filename):
    ds = pydicom.dcmread(dicomimage)
    img = apply_voi_lut(ds.pixel_array, ds)
    orginal_shape = img.shape
    img = resize(img, img.shape, anti_aliasing=True)
    
    if ds.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
        
    img = (((img - np.min(img))/np.max(img))*255.0).astype(np.uint8) 
    with open(filename + dicomimage.split('/')[-1].replace(".dicom", ".png"), "wb") as fn:
        img = cv2.imencode('.png',img)[1]
        data_encode = np.array(img)
        str_encode = data_encode.tostring()
        fn.write(str_encode)

In [ ]:
test_path = "../input/vinbigdata-chest-xray-abnormalities-detection/test/"

In [ ]:
test_images = os.listdir(test_path)

test_path_image = [test_path + image for image in test_images]

filetest = './original_testpng/' 
if not os.path.exists(os.path.dirname(filetest)):
    os.makedirs(filetest)

out = Parallel(n_jobs=-1)(delayed(convert_to_png)(image_path, filetest) \
    for image_path in tqdm(test_path_image))

In [ ]:
#First Binary classifier model
tez_path = '../input/tez-lib/'
timm_path = '../input/timm-pytorch-image-models/pytorch-image-models-master'
import sys
sys.path.append(tez_path)
sys.path.append(timm_path)

import os
import albumentations
import pandas as pd

import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image

from sklearn import metrics, model_selection, preprocessing
from scipy.special import softmax
import timm

IMG_SIZE = 512

class CathModel(tez.Model):
    def __init__(self, num_classes, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()

        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.out = nn.Linear(n_features, num_classes)
        self.step_scheduler_after = "epoch"
        self.criterion = nn.CrossEntropyLoss()
        self.transform = transforms.Compose([
            transforms.Resize((IMG_SIZE,IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
    
    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        outputs = self.model(image)
        return outputs, None, None
    
    def load(self, model_path, device="cuda"):
        self.device = device
        if next(self.parameters()).device != self.device:
            self.to(self.device)
        model_dict = torch.load(model_path, map_location=device)
        self.load_state_dict(model_dict["state_dict"])
        
    def predict(self, data, device="cuda"):
        data = self.transform(data).to(device)
        data = data.unsqueeze(0)
        with torch.no_grad():
            out = self.predict_one_step(data)
            out = out.cpu().detach().numpy()
            return out
        
    def model_fn(self, data):
        #for key, value in data.items():
        data = data.to(self.device)
        output = self.model(data)
        print(output)
        output = softmax(output)
        output = torch.argmax(output)
        return output
    
    def predict_one_step(self, data):
        output = self.model_fn(data)
        return output

# **Binary Classifier**

In [ ]:
model_BC = CathModel(num_classes=2, model_name='resnet18d')
model_BC.load("../input/detectionmodel/resnet18d.bin", device='cpu')
#img = Image.open("./original_testpng/f5f77203f00504c40b45a65d23cd8948.png").convert('RGB')
#out = model.predict(img,'cpu')

# 14 Class classifier

In [ ]:
model_14 = CathModel(num_classes=14, model_name='resnest101e')
model_14.load("../input/detectionmodel/resnest101e.bin", device='cuda')
#img = Image.open("./original_testpng/f5f77203f00504c40b45a65d23cd8948.png").convert('RGB')
#out = model_14.predict(img)
#out

# Detr Module

In [ ]:
import albumentations as A
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=False)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)
    
def predict(model, data, img,device='cuda'):
    model.eval()
    model.to(device)
    cpu_device = torch.device('cpu')
    h,w = img.size
    with torch.no_grad():
        outputs = model(data)
        
    outputs = [{k: v.to(cpu_device) for k, v in outputs.items()}]
    oboxes = outputs[0]['pred_boxes'][0].detach().cpu().numpy()
    oboxes = [np.array(box).astype(np.int32) for box in A.augmentations.bbox_utils.denormalize_bboxes(oboxes,h,w)]
    prob   = outputs[0]['pred_logits'][0].softmax(1).detach().cpu().numpy()[:,0]
    return oboxes, prob

detr_transform = transforms.Compose([
            transforms.Resize((IMG_SIZE,IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
#model_detr = DETRModel(num_classes=1,num_queries=100)
#model_detr.load_state_dict(torch.load("../input/detectionmodel/detrv1.pth", map_location='cpu'))

In [ ]:
#img = Image.open("./original_testpng/f5f77203f00504c40b45a65d23cd8948.png").convert('RGB')
#data = detr_transform(img).to('cpu')
#data = data.unsqueeze(0)
#predict(model_detr, data, img, 'cpu')

In [ ]:
import torchvision.transforms as transforms
import cv2
import numpy
import numpy as np
import glob

import torchvision
import torch

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

COLORS = np.random.uniform(0, 255, size=(2, 3))

# define the torchvision image transforms
transform = transforms.Compose([
    #transforms.Resize((IMG_SIZE*2,IMG_SIZE*2)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

coco_names = [
    '__background__',
    'Interest'
]

def predict(image, model, device, detection_threshold):
    # transform the image to tensor
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image) # get the predictions on the image
    # print the results individually
    # print(f"BOXES: {outputs[0]['boxes']}")
    # print(f"LABELS: {outputs[0]['labels']}")
    # print(f"SCORES: {outputs[0]['scores']}")
    # get all the predicited class names
    pred_classes = [coco_names[i] for i in outputs[0]['labels'].cpu().numpy()]
    # get score for all the predicted objects
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    # get all the predicted bounding boxes
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
    # get boxes above the threshold score
    boxes = pred_bboxes[pred_scores >= detection_threshold].astype(np.int32)
    return boxes, pred_classes, outputs[0]['labels'], pred_scores

def draw_boxes(boxes, classes, labels, image):
    # read the image with OpenCV
    image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2RGB)
    for i, box in enumerate(boxes):
        color = COLORS[labels[i]]
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 2
        )
        cv2.putText(image, classes[i], (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
                    lineType=cv2.LINE_AA)
    return image

In [ ]:
# download or load the model from disk
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load('../input/detectionmodel/frcnnv1.bin', map_location = device))
model.eval()

x = model.to(device)
from matplotlib import pyplot as plt

#for imgPath in glob.glob("./original_testpng/*"):
#    image = Image.open(imgPath).convert('RGB')
#    model.eval().to(device)
#    boxes, classes, labels, pred_scores = predict(image, model, device, 0.3)
    #image = draw_boxes(boxes, classes, labels, image)
#    print(imgPath, boxes)

#cv2.imshow('Image', image)
#cv2.imwrite(f"result.jpg", image)
#cv2.waitKey(0)


In [ ]:
def format_prediction_string(labels, boxes, scores):
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append("{0} {1:.4f} {2} {3} {4} {5}".format(
            j[0], j[1], j[2][0], j[2][1], j[2][2], j[2][3]))

    return " ".join(pred_strings)

In [ ]:
def finetune_classifier(boxes, img):
    labels = list()
    for bb in boxes:
        #img_crop = img[int(bb[1]):int(bb[3]),int(bb[0]):int(bb[2])]
        img_crop = img.crop((bb[0], bb[1], bb[2], bb[3]))
        out = 1 #model_14.predict(img_crop)
        #out = out.tolist()
        labels.append(out)
        
    return labels

In [ ]:
from tqdm.notebook import tqdm 
device = 'cuda'
results = list()
for imgPath in tqdm(glob.glob("./original_testpng/*")):
    image_id = imgPath.split("/")[-1].split(".")[0]
    result = {
    'image_id': image_id,
    'PredictionString': '14 1.0 0 0 1 1'
    }
    #Binary classifier
    img = Image.open(imgPath).convert('RGB')
    out = model_BC.predict(img,device)
    
    if out == 0:
        print("Empty case")
    else:
        #Pass into frcnn network
        boxes, classes, labels, pred_scores = predict(img, model, device, 0.3)
        if len(boxes):
            class_labels = finetune_classifier(boxes, img)
            result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(class_labels, boxes.astype(np.int32), pred_scores)
        }
        
    results.append(result)


In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()
